## Import package

In [ ]:
import csv
import gurobipy as gp
from gurobipy import GRB
import numpy as np

## Define constants

In [ ]:
dataset = 95 # <--- CHANGE THIS TO CHANGE DATASET
mipgap_number = 4

In [ ]:
student_hash_map = {}

with open(f'data_{dataset}_students.csv', newline='', encoding="utf-8") as csvfile:
    spamreader = list(csv.reader(csvfile, delimiter=','))
    spamreader.pop(0)
    for row in spamreader:
        try:
            student_hash_map[row[0]].append(row[1])
        except KeyError:
            student_hash_map[row[0]] = [row[1]]

exam_code_hash_map = {}

with open(f'data_{dataset}_exams_code.csv', newline='', encoding="utf-8") as csvfile:
    spamreader = list(csv.reader(csvfile, delimiter=','))
    spamreader.pop(0)
    for row in spamreader:
        exam_code_hash_map[row[0].strip()] = 0

for i in student_hash_map:
    for j in student_hash_map[i]:
        exam_code_hash_map[j] += 1

exam_time_hash_map = {}

exam_code_to_integer_hash_map = {}

# Map exam code to integer, for array indexing
counter = 0
for i in exam_code_hash_map:
    exam_code_to_integer_hash_map[i] = counter
    counter += 1
exam_student_hash_map = {}

# count how many student taking that exam
for i in student_hash_map:
    for j in student_hash_map[i]:
        try:
            exam_student_hash_map[str(exam_code_to_integer_hash_map[j])] += 1
        except KeyError:
            exam_student_hash_map[str(exam_code_to_integer_hash_map[j])] = 1

with open(f'data_{dataset}_exams_code.csv', newline='', encoding="utf-8") as csvfile:
    spamreader = list(csv.reader(csvfile, delimiter=','))
    spamreader.pop(0)
    for row in spamreader:
        temp = row[2].split(":")
        # exam time in hour
        exam_time = int(temp[0]) + int(temp[1]) / 60
        exam_time_hash_map[str(exam_code_to_integer_hash_map[row[0].strip()])] = exam_time
        


In [ ]:
exam_time_hash_map

In [ ]:
exam_rooms = [125, 80, 40, 40, 50, 35, 30, 25, 200, 160, 95, 480, 270] # together

In [ ]:
exam_code_to_integer_hash_map

In [ ]:
# Distinct exam timetable
reg_course_type_array = []
for i in student_hash_map:
    if student_hash_map[i] not in reg_course_type_array:
        reg_course_type_array.append(student_hash_map[i])

for i in reg_course_type_array:
    print(i)

In [ ]:
student_hash_map = {}

for i in range(len(reg_course_type_array)):
    student_hash_map[str(i)] = reg_course_type_array[i]

student_hash_map

In [ ]:
time_slot_different_array = []
for i in range(12):
    time_slot_different_array.append([])
    for j in range(12):
        if i > j:
            time_slot_different_array[i].append(int(1000*((i-j)**(1/2))))
        else:
            time_slot_different_array[i].append(0)

time_slot_different_array

In [ ]:
student_hash_map

## Model

### Initialize

In [ ]:
number_of_exam = len(list(exam_code_hash_map.keys()))
time_slot = 32 # 2 * ( 5*3 + 1 )
number_of_rooms = len(exam_rooms)

m = gp.Model()
m.Params.MIPGAP = mipgap_number
m.Params.MIPFocus = 3


x = m.addVars(number_of_exam, time_slot, name = "x", vtype=GRB.BINARY)

### Objective Function

In [ ]:
t = m.addVars(number_of_exam, 12, name="t", vtype=GRB.BINARY)

m.setObjective(sum(
    t[exam_code_to_integer_hash_map[h], p] * t[exam_code_to_integer_hash_map[j], k] * time_slot_different_array[p][k]
                   for i in student_hash_map
                   for p in range(12) 
                   for k in range(12) 
                   for h in student_hash_map[i] 
                   for j in student_hash_map[i] 
                   ), GRB.MAXIMIZE)

m.addConstrs(
(t[i,0] == sum(x[i,k] for k in range(0,3)) for i in range(number_of_exam)), 
name = "map_time_slot_to_day"
)
m.addConstrs(
(t[i,1] == sum(x[i,k] for k in range(3,6)) for i in range(number_of_exam)), 
name = "map_time_slot_to_day"
)
m.addConstrs(
(t[i,2] == sum(x[i,k] for k in range(6,9)) for i in range(number_of_exam)), 
name = "map_time_slot_to_day"
)
m.addConstrs(
(t[i,3] == sum(x[i,k] for k in range(9,12)) for i in range(number_of_exam)), 
name = "map_time_slot_to_day"
)
m.addConstrs(
(t[i,4] == sum(x[i,k] for k in range(12,15)) for i in range(number_of_exam)), 
name = "map_time_slot_to_day"
)
m.addConstrs(
(t[i,5] == sum(x[i,k] for k in range(15,16)) for i in range(number_of_exam)), 
name = "map_time_slot_to_day"
)
m.addConstrs(
(t[i,6] == sum(x[i,k] for k in range(16,19)) for i in range(number_of_exam)), 
name = "map_time_slot_to_day"
)
m.addConstrs(
(t[i,7] == sum(x[i,k] for k in range(19,22)) for i in range(number_of_exam)), 
name = "map_time_slot_to_day"
)
m.addConstrs(
(t[i,8] == sum(x[i,k] for k in range(22,25)) for i in range(number_of_exam)), 
name = "map_time_slot_to_day"
)
m.addConstrs(
(t[i,9] == sum(x[i,k] for k in range(25,28)) for i in range(number_of_exam)), 
name = "map_time_slot_to_day"
)
m.addConstrs(
(t[i,10] == sum(x[i,k] for k in range(28,31)) for i in range(number_of_exam)), 
name = "map_time_slot_to_day"
)
m.addConstrs(
(t[i,11] == sum(x[i,k] for k in range(31,32)) for i in range(number_of_exam)), 
name = "map_time_slot_to_day"
)

### Constraint: exam duration

In [ ]:
# exam duration constraints
m.addConstrs( ((x[i,j] * exam_time_hash_map[str(i)]) <= 3 for i in range(number_of_exam) for j in [0,3,6,9,12,15,16,19,22,25,28,31]), name="exam_duration_constraints")
m.addConstrs( ((x[i,j] * exam_time_hash_map[str(i)]) <= 2 for i in range(number_of_exam) for j in [1,2,4,5,7,8,10,11,13,14,17,18,20,21,23,24,26,27,29,30]), name="exam_duration_constraints")

### Constraint: room capacity

In [ ]:
y = m.addVars(number_of_exam, time_slot, number_of_rooms, name = "exam_room", vtype = GRB.BINARY)

for k in range(number_of_rooms):
    m.addConstrs((y[i, j, k] <= x[i, j] for i in range(number_of_exam) for j in range(time_slot)), name = f"that exam that time slot must have room") # need <=, not ==, or else all exam room need to == 1

m.addConstrs((sum(y[i, j, k] for j  in range(time_slot) for k in range(number_of_rooms)) == 1 for i in range(number_of_exam)), name = f"exam_must_hold_in_one_room")

m.addConstrs((sum(y[i, j, k] for i in range(number_of_exam)) <= 1 for k in range(number_of_rooms) for j in range(time_slot)), name = f"each room each time slot only one exam")
m.addConstrs((sum(y[i, j, k] * exam_student_hash_map[str(i)] for i in range(number_of_exam)) <= exam_rooms[k] for k in range(number_of_rooms) for j in range(time_slot)), name = f"exam capacity <= room capacity")


### Constraint: no same exam at same time

In [ ]:
# Avoid the same student from taking multiple exams simultaneously.
for i in student_hash_map: 
    m.addConstrs(
        (sum(x[exam_code_to_integer_hash_map[j], k] for j in student_hash_map[i]) <= 1 for k in range(time_slot))
        , name = f"no same exam at same time {i}")

### Constraint: no two exam at same day

In [ ]:
for i in student_hash_map:
    m.addConstrs(
        (
        (sum(x[exam_code_to_integer_hash_map[j], k] for j in student_hash_map[i])
        + sum(x[exam_code_to_integer_hash_map[j], k+1] for j in student_hash_map[i]) 
        + sum(x[exam_code_to_integer_hash_map[j], k+2] for j in student_hash_map[i])
        )<= 2
        for k in range(0,1)
        )
        , name = f"no_two_exam_at_same_day1_{i}"
    )
    m.addConstrs(
        (
        (sum(x[exam_code_to_integer_hash_map[j], k] for j in student_hash_map[i])
        + sum(x[exam_code_to_integer_hash_map[j], k+1] for j in student_hash_map[i]) 
        + sum(x[exam_code_to_integer_hash_map[j], k+2] for j in student_hash_map[i])
        )<= 2
        for k in range(3,4)
        )
        , name = f"no_two_exam_at_same_day2_{i}"
    )
    m.addConstrs(
        (
        (sum(x[exam_code_to_integer_hash_map[j], k] for j in student_hash_map[i])
        + sum(x[exam_code_to_integer_hash_map[j], k+1] for j in student_hash_map[i]) 
        + sum(x[exam_code_to_integer_hash_map[j], k+2] for j in student_hash_map[i])
        )<= 2
        for k in range(6,7)
        )
        , name = f"no_two_exam_at_same_day3_{i}"
    )
    m.addConstrs(
        (
        (sum(x[exam_code_to_integer_hash_map[j], k] for j in student_hash_map[i])
        + sum(x[exam_code_to_integer_hash_map[j], k+1] for j in student_hash_map[i]) 
        + sum(x[exam_code_to_integer_hash_map[j], k+2] for j in student_hash_map[i])
        )<= 2
        for k in range(9,10)
        )
        , name = f"no_two_exam_at_same_day4_{i}"
    )
    m.addConstrs(
        (
        (sum(x[exam_code_to_integer_hash_map[j], k] for j in student_hash_map[i])
        + sum(x[exam_code_to_integer_hash_map[j], k+1] for j in student_hash_map[i]) 
        + sum(x[exam_code_to_integer_hash_map[j], k+2] for j in student_hash_map[i])
        )<= 2
        for k in range(12,13)
        )
        , name = f"no_two_exam_at_same_day5_{i}"
    )
    m.addConstrs(
        (
        (sum(x[exam_code_to_integer_hash_map[j], k] for j in student_hash_map[i])
        )<= 2
        for k in range(15,16)
        )
        , name = f"no_two_exam_at_same_day6_{i}"
    )
    m.addConstrs(
        (
        (sum(x[exam_code_to_integer_hash_map[j], k] for j in student_hash_map[i])
        + sum(x[exam_code_to_integer_hash_map[j], k+1] for j in student_hash_map[i]) 
        + sum(x[exam_code_to_integer_hash_map[j], k+2] for j in student_hash_map[i])
        )<= 2
        for k in range(16,17)
        )
        , name = f"no_two_exam_at_same_day7_{i}"
    )
    m.addConstrs(
        (
        (sum(x[exam_code_to_integer_hash_map[j], k] for j in student_hash_map[i])
        + sum(x[exam_code_to_integer_hash_map[j], k+1] for j in student_hash_map[i]) 
        + sum(x[exam_code_to_integer_hash_map[j], k+2] for j in student_hash_map[i])
        )<= 2
        for k in range(19,20)
        )
        , name = f"no_two_exam_at_same_day8_{i}"
    )
    m.addConstrs(
        (
        (sum(x[exam_code_to_integer_hash_map[j], k] for j in student_hash_map[i])
        + sum(x[exam_code_to_integer_hash_map[j], k+1] for j in student_hash_map[i]) 
        + sum(x[exam_code_to_integer_hash_map[j], k+2] for j in student_hash_map[i])
        )<= 2
        for k in range(22,23)
        )
        , name = f"no_two_exam_at_same_day9_{i}"
    )
    m.addConstrs(
        (
        (sum(x[exam_code_to_integer_hash_map[j], k] for j in student_hash_map[i])
        + sum(x[exam_code_to_integer_hash_map[j], k+1] for j in student_hash_map[i]) 
        + sum(x[exam_code_to_integer_hash_map[j], k+2] for j in student_hash_map[i])
        )<= 2
        for k in range(25,26)
        )
        , name = f"no_two_exam_at_same_day10_{i}"
    )
    m.addConstrs(
        (
        (sum(x[exam_code_to_integer_hash_map[j], k] for j in student_hash_map[i])
        + sum(x[exam_code_to_integer_hash_map[j], k+1] for j in student_hash_map[i]) 
        + sum(x[exam_code_to_integer_hash_map[j], k+2] for j in student_hash_map[i])
        )<= 2
        for k in range(28,29)
        )
        , name = f"no_two_exam_at_same_day11_{i}"
    )
    m.addConstrs(
        (
        (sum(x[exam_code_to_integer_hash_map[j], k] for j in student_hash_map[i])
        )<= 2
        for k in range(31,32)
        )
        , name = f"no_two_exam_at_same_day12_{i}"
    )

### Constraint: no consecutive exam for student

In [ ]:
for i in student_hash_map:
    m.addConstrs(
        (
        (sum(x[exam_code_to_integer_hash_map[j], k] for j in student_hash_map[i])
        + sum(x[exam_code_to_integer_hash_map[j], k+1] for j in student_hash_map[i]) 
        )<= 1
        for k in range(0,1)
        )
        , name = f"no_consecutive_{i}"
    )
    m.addConstrs(
        (
        (sum(x[exam_code_to_integer_hash_map[j], k] for j in student_hash_map[i])
        + sum(x[exam_code_to_integer_hash_map[j], k+1] for j in student_hash_map[i]) 
        )<= 1
        for k in range(1,2)
        )
        , name = f"no_consecutive_{i}"
    )
    m.addConstrs(
        (
        (sum(x[exam_code_to_integer_hash_map[j], k] for j in student_hash_map[i])
        + sum(x[exam_code_to_integer_hash_map[j], k+1] for j in student_hash_map[i]) 
        )<= 1
        for k in range(2,3)
        )
        , name = f"no_consecutive_{i}"
    )
    m.addConstrs(
        (
        (sum(x[exam_code_to_integer_hash_map[j], k] for j in student_hash_map[i])
        + sum(x[exam_code_to_integer_hash_map[j], k+1] for j in student_hash_map[i]) 
        )<= 1
        for k in range(3,4)
        )
        , name = f"no_consecutive_{i}"
    )
    m.addConstrs(
        (
        (sum(x[exam_code_to_integer_hash_map[j], k] for j in student_hash_map[i])
        + sum(x[exam_code_to_integer_hash_map[j], k+1] for j in student_hash_map[i]) 
        )<= 1
        for k in range(4,5)
        )
        , name = f"no_consecutive_{i}"
    )
    m.addConstrs(
        (
        (sum(x[exam_code_to_integer_hash_map[j], k] for j in student_hash_map[i])
        + sum(x[exam_code_to_integer_hash_map[j], k+1] for j in student_hash_map[i]) 
        )<= 1
        for k in range(5,6)
        )
        , name = f"no_consecutive_{i}"
    )
    m.addConstrs(
        (
        (sum(x[exam_code_to_integer_hash_map[j], k] for j in student_hash_map[i])
        + sum(x[exam_code_to_integer_hash_map[j], k+1] for j in student_hash_map[i]) 
        )<= 1
        for k in range(6,7)
        )
        , name = f"no_consecutive_{i}"
    )
    m.addConstrs(
        (
        (sum(x[exam_code_to_integer_hash_map[j], k] for j in student_hash_map[i])
        + sum(x[exam_code_to_integer_hash_map[j], k+1] for j in student_hash_map[i]) 
        )<= 1
        for k in range(7,8)
        )
        , name = f"no_consecutive_{i}"
    )
    m.addConstrs(
        (
        (sum(x[exam_code_to_integer_hash_map[j], k] for j in student_hash_map[i])
        + sum(x[exam_code_to_integer_hash_map[j], k+1] for j in student_hash_map[i]) 
        )<= 1
        for k in range(8,9)
        )
        , name = f"no_consecutive_{i}"
    )
    m.addConstrs(
        (
        (sum(x[exam_code_to_integer_hash_map[j], k] for j in student_hash_map[i])
        + sum(x[exam_code_to_integer_hash_map[j], k+1] for j in student_hash_map[i]) 
        )<= 1
        for k in range(9,10)
        )
        , name = f"no_consecutive_{i}"
    )
    m.addConstrs(
        (
        (sum(x[exam_code_to_integer_hash_map[j], k] for j in student_hash_map[i])
        + sum(x[exam_code_to_integer_hash_map[j], k+1] for j in student_hash_map[i]) 
        )<= 1
        for k in range(10,11)
        )
        , name = f"no_consecutive_{i}"
    )
    m.addConstrs(
        (
        (sum(x[exam_code_to_integer_hash_map[j], k] for j in student_hash_map[i])
        + sum(x[exam_code_to_integer_hash_map[j], k+1] for j in student_hash_map[i]) 
        )<= 1
        for k in range(11,12)
        )
        , name = f"no_consecutive_{i}"
    )
    m.addConstrs(
        (
        (sum(x[exam_code_to_integer_hash_map[j], k] for j in student_hash_map[i])
        + sum(x[exam_code_to_integer_hash_map[j], k+1] for j in student_hash_map[i]) 
        )<= 1
        for k in range(12,13)
        )
        , name = f"no_consecutive_{i}"
    )
    m.addConstrs(
        (
        (sum(x[exam_code_to_integer_hash_map[j], k] for j in student_hash_map[i])
        + sum(x[exam_code_to_integer_hash_map[j], k+1] for j in student_hash_map[i]) 
        )<= 1
        for k in range(13,14)
        )
        , name = f"no_consecutive_{i}"
    )
    m.addConstrs(
        (
        (sum(x[exam_code_to_integer_hash_map[j], k] for j in student_hash_map[i])
        + sum(x[exam_code_to_integer_hash_map[j], k+1] for j in student_hash_map[i]) 
        )<= 1
        for k in range(14,15)
        )
        , name = f"no_consecutive_{i}"
    )
    m.addConstrs(
        (
        (sum(x[exam_code_to_integer_hash_map[j], k] for j in student_hash_map[i])
        + sum(x[exam_code_to_integer_hash_map[j], k+1] for j in student_hash_map[i]) 
        )<= 1
        for k in range(15,16)
        )
        , name = f"no_consecutive_{i}"
    )
    m.addConstrs(
        (
        (sum(x[exam_code_to_integer_hash_map[j], k] for j in student_hash_map[i])
        + sum(x[exam_code_to_integer_hash_map[j], k+1] for j in student_hash_map[i]) 
        )<= 1
        for k in range(16,17)
        )
        , name = f"no_consecutive_{i}"
    )
    m.addConstrs(
        (
        (sum(x[exam_code_to_integer_hash_map[j], k] for j in student_hash_map[i])
        + sum(x[exam_code_to_integer_hash_map[j], k+1] for j in student_hash_map[i]) 
        )<= 1
        for k in range(17,18)
        )
        , name = f"no_consecutive_{i}"
    )
    m.addConstrs(
        (
        (sum(x[exam_code_to_integer_hash_map[j], k] for j in student_hash_map[i])
        + sum(x[exam_code_to_integer_hash_map[j], k+1] for j in student_hash_map[i]) 
        )<= 1
        for k in range(18,19)
        )
        , name = f"no_consecutive_{i}"
    )
    m.addConstrs(
        (
        (sum(x[exam_code_to_integer_hash_map[j], k] for j in student_hash_map[i])
        + sum(x[exam_code_to_integer_hash_map[j], k+1] for j in student_hash_map[i]) 
        )<= 1
        for k in range(19,20)
        )
        , name = f"no_consecutive_{i}"
    )
    m.addConstrs(
        (
        (sum(x[exam_code_to_integer_hash_map[j], k] for j in student_hash_map[i])
        + sum(x[exam_code_to_integer_hash_map[j], k+1] for j in student_hash_map[i]) 
        )<= 1
        for k in range(20,21)
        )
        , name = f"no_consecutive_{i}"
    )
    m.addConstrs(
        (
        (sum(x[exam_code_to_integer_hash_map[j], k] for j in student_hash_map[i])
        + sum(x[exam_code_to_integer_hash_map[j], k+1] for j in student_hash_map[i]) 
        )<= 1
        for k in range(21,22)
        )
        , name = f"no_consecutive_{i}"
    )
    m.addConstrs(
        (
        (sum(x[exam_code_to_integer_hash_map[j], k] for j in student_hash_map[i])
        + sum(x[exam_code_to_integer_hash_map[j], k+1] for j in student_hash_map[i]) 
        )<= 1
        for k in range(22,23)
        )
        , name = f"no_consecutive_{i}"
    )
    m.addConstrs(
        (
        (sum(x[exam_code_to_integer_hash_map[j], k] for j in student_hash_map[i])
        + sum(x[exam_code_to_integer_hash_map[j], k+1] for j in student_hash_map[i]) 
        )<= 1
        for k in range(23,24)
        )
        , name = f"no_consecutive_{i}"
    )
    m.addConstrs(
        (
        (sum(x[exam_code_to_integer_hash_map[j], k] for j in student_hash_map[i])
        + sum(x[exam_code_to_integer_hash_map[j], k+1] for j in student_hash_map[i]) 
        )<= 1
        for k in range(24,25)
        )
        , name = f"no_consecutive_{i}"
    )
    m.addConstrs(
        (
        (sum(x[exam_code_to_integer_hash_map[j], k] for j in student_hash_map[i])
        + sum(x[exam_code_to_integer_hash_map[j], k+1] for j in student_hash_map[i]) 
        )<= 1
        for k in range(25,26)
        )
        , name = f"no_consecutive_{i}"
    )
    m.addConstrs(
        (
        (sum(x[exam_code_to_integer_hash_map[j], k] for j in student_hash_map[i])
        + sum(x[exam_code_to_integer_hash_map[j], k+1] for j in student_hash_map[i]) 
        )<= 1
        for k in range(26,27)
        )
        , name = f"no_consecutive_{i}"
    )
    m.addConstrs(
        (
        (sum(x[exam_code_to_integer_hash_map[j], k] for j in student_hash_map[i])
        + sum(x[exam_code_to_integer_hash_map[j], k+1] for j in student_hash_map[i]) 
        )<= 1
        for k in range(27,28)
        )
        , name = f"no_consecutive_{i}"
    )
    m.addConstrs(
        (
        (sum(x[exam_code_to_integer_hash_map[j], k] for j in student_hash_map[i])
        + sum(x[exam_code_to_integer_hash_map[j], k+1] for j in student_hash_map[i]) 
        )<= 1
        for k in range(28,29)
        )
        , name = f"no_consecutive_{i}"
    )
    m.addConstrs(
        (
        (sum(x[exam_code_to_integer_hash_map[j], k] for j in student_hash_map[i])
        + sum(x[exam_code_to_integer_hash_map[j], k+1] for j in student_hash_map[i]) 
        )<= 1
        for k in range(29,30)
        )
        , name = f"no_consecutive_{i}"
    )
    m.addConstrs(
        (
        (sum(x[exam_code_to_integer_hash_map[j], k] for j in student_hash_map[i])
        + sum(x[exam_code_to_integer_hash_map[j], k+1] for j in student_hash_map[i]) 
        )<= 1
        for k in range(30,31)
        )
        , name = f"no_consecutive_{i}"
    )

### Constraint: exam must hold once

In [ ]:
# sum X [exam, all time slot] == 1
m.addConstrs((sum(x[j, k] for k in range(time_slot)) == 1 for j in range(number_of_exam)), name = f"exam_must_hold_once")

### Constraint: special condition

In [ ]:
# Special condidition: these exam code not in 22 and 95
if dataset == None:
    m.addConstr(( x[exam_code_to_integer_hash_map["F321Q6E1"], 13]+x[exam_code_to_integer_hash_map["F321Q6E1"], 14]+ x[exam_code_to_integer_hash_map["F321Q6E1"], 15] )==1, name = "F321Q6E1 27th Jan")

    m.addConstr( (x[exam_code_to_integer_hash_map["F321T6E1"], 20]+x[exam_code_to_integer_hash_map["F321T6E1"], 21]+ x[exam_code_to_integer_hash_map["F321T6E1"], 22] ) ==1, name = "F321T6E1 30th Jan")

    m.addConstr(( x[exam_code_to_integer_hash_map["H21M01E1"], 1]+x[exam_code_to_integer_hash_map["H21M01E1"], 2]+ x[exam_code_to_integer_hash_map["H21M01E1"], 3]+ x[exam_code_to_integer_hash_map["H21M01E1"], 4]+x[exam_code_to_integer_hash_map["H21M01E1"], 5]+x[exam_code_to_integer_hash_map["H21M01E1"], 6] )==1, name = "H21M01E1 23rd-24th Jan")

    m.addConstr( (x[exam_code_to_integer_hash_map["H22M02E1"], 1]+x[exam_code_to_integer_hash_map["H22M02E1"], 4]+x[exam_code_to_integer_hash_map["H22M02E1"], 2]+ x[exam_code_to_integer_hash_map["H22M02E1"], 3]+x[exam_code_to_integer_hash_map["H22M02E1"], 5]+x[exam_code_to_integer_hash_map["H22M02E1"], 6] ) ==1, name = "H22M02E1 23rd-24th Jan")



    m.addConstr((x[exam_code_to_integer_hash_map["H2CM04E1"], 1]+x[exam_code_to_integer_hash_map["H2CM04E1"], 4]+x[exam_code_to_integer_hash_map["H2CM04E1"], 2]+ x[exam_code_to_integer_hash_map["H2CM04E1"], 3]+x[exam_code_to_integer_hash_map["H2CM04E1"], 5]+x[exam_code_to_integer_hash_map["H2CM04E1"], 6] )==1, name = "H2CM04E1 23rd-24th Jan")

    m.addConstr(  sum(  x[exam_code_to_integer_hash_map["G13RE2E1"], k] for k in range(19)  )  == 1 , name = "G13RE2E1  before 30th Jan")
    lst = [1,4,7,10,13,16,17,20,23,26,29,32]
    m.addConstr(  sum(  x[exam_code_to_integer_hash_map["K1AHWAE2"], k]  for k in range(len(lst))  )  == 1, name = "K1AHWAE2  any am slot")
    m.addConstr(  sum(  x[exam_code_to_integer_hash_map["H63122E1"], k]  for k in range(len(lst)) )  == 1 , name = "H63122E1  any am slot")
    lst = [10,11,12,26,27,28]
    m.addConstr(  sum(  x[exam_code_to_integer_hash_map["V13101E1"], k]  for k in range(len(lst)) )  == 1 , name = "V13101E1  any am slot")

    #F13P03E1 \ before / F13X03E1#
    for k in range(32):
        for j in range(32):
            if k < j:
                m.addConstr( ( x[exam_code_to_integer_hash_map["F13P03E1"], k] * x[exam_code_to_integer_hash_map["F13X03E1"], j]  <= 1   ))
                #print(str(k) + "  "+ str(j))

    #F13P05E1 \ before / F13X04E1#
    for k in range(32):
        for j in range(32):
            if k < j:
                m.addConstr(( x[exam_code_to_integer_hash_map["F13P05E1"], k] * x[exam_code_to_integer_hash_map["F13X04E1#"], j]   <= 1  ))
                
    #H3BFM2E1  must be immediately followed by  H3BFM2E2#
    for k in range(31):
        m.addConstr(  (x[exam_code_to_integer_hash_map["H3BFM2E1"], k] * x[exam_code_to_integer_hash_map["H3BFM2E2"], k+1]    <= 1 ))

    #H8B040E1 must be at different times H8C001E1#
    for k in range(32):
        for j in range(32):
            if k != j:
                m.addConstr( ( x[exam_code_to_integer_hash_map["H8B040E1"], k] * x[exam_code_to_integer_hash_map["H8C001E1"], j]   <= 1  ))

## Result

In [ ]:
m.write("test.lp")

In [ ]:
m.optimize()

In [ ]:
# The number of students in each exam cannot exceed the capacity of the venue.


print("==============================================")
for i in range(number_of_exam):
    for j in range(time_slot):
        if x[i,j].x == 1:
            print(list(exam_code_hash_map.keys())[i], j)

print("==============================================")
print("exam_code \t ime_slot \t room \t how_many_student_take_that_exam \t room_capacity")
for i in range(number_of_exam):
    for j in range(time_slot):
        for k in range(number_of_rooms):
            if y[i,j,k].x == 1:
                print(list(exam_code_hash_map.keys())[i], "\t", j, "\t", k, "\t", exam_student_hash_map[str(i)], "\t", exam_rooms[k])